In [2]:
import os
import pandas as pd
import numpy as np

def get_title_list(path):
    """記事タイトル取得関数"""
    title_list = []
    filenames = os.listdir(path) #ファイル名称一覧取得
    for filename in filenames:
        # 1記事ずつファイルの読み込み
        with open(path+filename, encoding="utf_8_sig") as f:
            title = f.readlines()[2].strip() #各記事テキストの改行2番目に記事タイトルが記載してある
            title_list.append(title)
    return title_list

# データセットの生成(タイトルとラベル付与)
"""
今回は例として与えられた記事タイトルから
どのニュース媒体記事なのか？(独女通信、ITライフハック、MOVIE ENTERの3種類)
を分類する為のデータセットを作成する
"""

#独女通信(ラベル0)
# #ITライフハック(ラベル1)
# #MOVIE ENTER(ラベル2)
title_0 = get_title_list('../text/dokujo-tsushin/')
label_0 = np.full(len(title_0),0)
title_1 = get_title_list('../text/it-life-hack/')
label_1 = np.full(len(title_1),1)
title_2 = get_title_list('../text/movie-enter/')
label_2 = np.full(len(title_2),2)

title_list = title_0 + title_1 + title_2
label_list = np.concatenate([label_0,label_1,label_2])

df = pd.DataFrame({
    'label': label_list,
    'title': title_list
})

# 全データの順番をシャッフル(+index振り直し)
df = df.sample(frac=1 ,random_state=0).reset_index(drop=True)
df.to_csv('livedoor_sentence.csv', sep=',', index=False, encoding='utf_8_sig')

In [ ]:
#google colab用
!pip install unidic-lite
!pip install fugashi

In [ ]:
import pandas as pd
df = pd.read_csv('livedoor_sentence.csv', index_col=None)

In [3]:
import collections
print(collections.Counter(df['label']))

Counter({1: 871, 2: 871, 0: 871})


In [4]:
from transformers import BertJapaneseTokenizer

"""
使用する前にfugashiとunidic-liteをインストールする。
pip install unidic-lite 
pip install fugashi

・BertJapaneseTokenizerは日本語用のBERTトークナイザ
・from_pretrainedで指定されたPytorchモデルの事前学習を実行する
・"cl-tohoku/bert-base-japanese-v2"は東北大学の日本語事前学習用BERTモデル ※下記補足参照
・do_subword_tokenizeは、サブワードのトークン化をするかどうか
・mecab_kwargsでMeCabユーザー辞書やNeoLogd等の指定も可能
　※"mecab_dic": Noneでデフォルト辞書(UniDic)をOFFにする必要あり
"""
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v2", 
    #do_subword_tokenize=False,
    #mecab_kwargs={"mecab_dic": None, "mecab_option": "-d 'C:\mecab-unidic-neologd'"
)

#適当なキーワードでトークナイズしてみる
text = "楽しくリズム感覚が身につく"

#tokenizer.encodeでテキストをトークンIDに,return_tensors='pt'でPytorch型のテンソル型に変換
ids = tokenizer.encode(text, return_tensors='pt')[0]
wakati = tokenizer.convert_ids_to_tokens(ids) #どのようにトークナイズされたか分かち書きで確認
print(ids)
print(wakati)


c:\Users\takig\.conda\envs\miko-py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\takig\.conda\envs\miko-py310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([    2, 32589, 17651, 16947,   862,  5128,   893, 12953,     3])
['[CLS]', '楽しく', 'リズム', '感覚', 'が', '身', 'に', 'つく', '[SEP]']


In [5]:
"""
・BertForSequenceClassificationはBEETの最終層をクラス分類に変えたもの
・事前学習はトークナイザと同じものを指定する
・num_labelsで分類数を指定する
"""
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-v2",
    num_labels = 3
)

#学習モードに設定
model.train()

#使用デバイス設定(CPU,GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) #modelをGPUに転送

#オプティマイザの設定
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\takig\.conda\envs\miko-py310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
#乱数のseedを全固定する
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(0)


In [8]:
from torch.utils.data import TensorDataset, random_split, DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

"""
・encodingで分類対象の文をトークナイザ
・input_idsはトークンID
・attention_mask はパディング用に埋め込み文字化どうかの判断用
・train_labels は分類ラベル
"""
encoding = tokenizer(df['title'].tolist(), return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
train_labels = torch.tensor(df['label'].tolist())

#データセット作成
dataset = TensorDataset(input_ids, attention_mask, train_labels)

#学習とテストの割合　※ここでは9割学習
train_size = int(0.9*len(dataset))
val_size= len(dataset) - train_size
train_dataset, val_dataset= random_split(dataset, [train_size, val_size])

print("学習データ数: {}" .format(train_size))
print("テストデータ数: {}" .format(val_size))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


学習データ数: 2351
テストデータ数: 262


In [10]:
import pandas as pd
import numpy as np

for i in range(val_dataset.__len__()):
    #別の方法あるかもだが、文字置換でスペシャルトークン(CLSとか)を消している
    sentence = tokenizer.decode(val_dataset.__getitem__(i)[0].detach().numpy().tolist()).replace('[CLS]', '').replace('[PAD]', '').replace('[SEP]', '').replace(' ', '')
    df_val_sentence_kari = pd.DataFrame(sentence.split(), columns=["title"])
        
    if i==0:
        df_val_sentence = df_val_sentence_kari.copy()
    else:
        df_val_sentence = pd.concat([df_val_sentence_kari, df_val_sentence], axis=0)

df_val_sentence.head(2) #お試しで2行表示


,title
0,もうプロポーズを待たない女たち
0,写真魂のバトンリレー!GRデジタルをバトンに若き写真家たちの駅伝写真展がスタート


In [11]:
batch_size = 8

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
#     sampler = RandomSampler(val_dataset), #先ほど元sentenceを保存した為シャッフルしない
    batch_size = 1
)


In [ ]:
def train(model):
    """学習ループ用関数"""
    model.train()
    train_loss = 0
    
    for batch in train_dataloader:
        b_input_ids = batch[0].to(device) #トークンID
        b_input_mask = batch[1].to(device) #埋め込み文字判断
        b_labels = batch[2].to(device) #正解ラベル
        optimizer.zero_grad()
        
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()    
    return train_loss

max_epoch = 10 #Epoch数は適当に
train_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    train_loss_.append(train_)
    
    print(train_)


In [ ]:
def eval(model, validation_dataloader, df_val_sentence) :
    model.eval() #検証モード
    preds_list = []
    b_labels_list = []

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device) #ラベルはモデルへ入力しないが正解確認用
        
        with torch.no_grad():
            preds = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            preds_list.append(preds) #予測ラベル
            b_labels_list.append(b_labels) #正解ラベル

    #データフレームで結果を可視化する
    for i in range(len(preds_list)):
        #preds_list[i][0]で一番確率が高いものを予測ラベルと判断(よくある分類のパターンと同じ)
        df_pred = pd.DataFrame(np.argmax(preds_list[i][0].cpu().numpy(), axis=1), columns={"pred_label"})
        df_label = pd.DataFrame(b_labels_list[i].cpu().numpy(), columns={"true_label"})
        df_result_kari = pd.concat([df_pred, df_label], axis=1)
        
        if i==0:
            df_result = df_result_kari.copy()
        else:
            df_result = pd.concat([df_result_kari, df_result], axis=0)

    df_result = pd.concat([df_val_sentence, df_result], axis=1) 
    return df_result


In [ ]:
plain = eval(model, validation_dataloader, df_val_sentence)
plain.head()